In [1]:
#import
import numpy as np
from scipy.misc import imread, imresize, imsave
from PIL import Image
import os
import glob

import tensorflow as tf
from tensorflow import keras
import matplotlib
matplotlib.use('TKAgg')

import matplotlib.pyplot as plt

import time
import random
from tqdm import tqdm, trange

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [ ]:
# https://blog.naver.com/whdgml1996/221283253466
# Data Preprocessing

x_train = tf.Session()
y_train = tf.Session()
x_test = tf.Session()
y_test = tf.Session()

# 폴더에서 파일 이름을 읽어서 이름에 따라 indexing
nameArr = ['소방차', '화재경', '비상벨'] # class를 정의하는 배열
tempArr = []
jpgList = glob.glob("*.jpg")
for i in range(len(jpgList)):
    for j in range(len(nameArr)):
        if (jpgList[i][0:3]==nameArr[j]):
            tempIndex = []
            tempIndex.append(j)
            tempArr.append(tempIndex)
            
print('y_train에 해당하는 one hot encoding할 배열', end = ' : ')
print(tempArr)

In [ ]:
"""
0. 실제로 데이터셋 많이 쌓기
1. train, test data set을 tensor로 만들기
2. Conv-ReLU-Pool-Conv-ReLLU-Pool-Conv-ReLU-Pool-Fully 모델 생성
3. Backpropagation
4. test data set으로 accuracy, loss 측정
"""

## ==============================

### tensorflow pdf로 실험중
https://github.com/Apress/pro-deep-learning-w-tensorflow/blob/master/9781484230954/Chapter3.ipynb

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import time

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/",one_hot=True)

In [ ]:
num_batches = mnist.train.num_examples/batch_size

In [ ]:
learning_rate = 0.01
epochs = 20
batch_size = 256
input_height = 28
input_width = 28
n_classes = 10
dropout = 0.75
display_step = 1
filter_height = 5
filter_width = 5
depth_in = 1
depth_out1 = 64
depth_out2 = 128

In [ ]:
x = tf.placeholder(tf.float32,[None,28*28])
y = tf.placeholder(tf.float32,[None,n_classes])
keep_prob = tf.placeholder(tf.float32)

weights = {
'wc1' : tf.Variable(tf.random_normal([filter_height,filter_width,depth_in,depth_out1])),
'wc2' : tf.Variable(tf.random_normal([filter_height,filter_width,depth_out1,depth_out2])),
'wd1' : tf.Variable(tf.random_normal([(input_height/4)*(input_height/4)* depth_out2,1024])),
'out' : tf.Variable(tf.random_normal([1024,n_classes]))
}

biases = {
'bc1' : tf.Variable(tf.random_normal([64])),
'bc2' : tf.Variable(tf.random_normal([128])),
'bd1' : tf.Variable(tf.random_normal([1024])),
'out' : tf.Variable(tf.random_normal([n_classes]))
}

In [ ]:
def conv2d(x, W, b, strides = 1):
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding = 'SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)

def maxpool2d(x, stride = 2):
    return tf.nn.max_pool(x, ksize = [1, stride, stride, 1], strides = [1, stride, stride, 1], padding = 'SAME')


In [ ]:
def conv_net(x, weights, biases, dropout):
    x = tf.reshape(x, shape = [-1, 28, 28, 1])
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    conv1 = maxpool2d(conv1, 2)
    
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    conv2 = maxpool2d(conv2, 2)
    
    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    
    fc1 = tf.nn.dropout(fc1, dropout)
    
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out

In [ ]:
pred = conv_net(x,weights,biases,keep_prob)
# Define loss function and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred,labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
# Evaluate model
correct_pred = tf.equal(tf.argmax(pred,1),tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred,tf.float32))
## initializing all variables
init = tf.global_variables_initializer()

### ref.0

In [ ]:
# https://blog.naver.com/whdgml1996/221283253466

# Data Preprocessing
nameArr = ['소방차', '화재경', '비상벨']

def one_hot(i):
    a = np.zeros(len(nameArr), 'uint8')
    a[i] = 1
    return a

jpgList = glob.glob("*.jpg")
x_train = []
y_train = []
for i in range(len(jpgList)):
    x_train.append(imread(jpgList[i]))
    for j in range(len(nameArr)):
        if(jpgList[i][0:3]==nameArr[j]):
            y_train.append(one_hot(j))

In [ ]:
learning_rate = 0.001
batch_size = 500
training_epochs = 500

# 64 64로 변환 - 방법1
#x_train = tf.reshape(x_train[0], shape = [64, 64, -1])
#print(x_train.shape)

# 64 64로 변환 - 방법2
#x_train = tf.placeholder("float", [None, 64, 64, 1])

In [ ]:
# test, train data set은 임의로 train을 복사
x_test = x_train
y_test = y_train

In [ ]:
# CRPCRPCRF -> Backpropagation -> print Score

In [ ]:
# http://blog.aloni.org/posts/backprop-with-tensorflow/
print('x_train shape : ', x_train.shape)
print('y_train shape : ', y_train.shape)
print('x_test shape : ', x_test.shape)
print('y_test shape : ', y_test.shape)

### ref.1

In [ ]:
# tf.nn.conv2d(input, filter, strides, padding, use_cudnn_on_gpu=None, data_format=None, name=None)
# https://cafe.naver.com/soynature/1234
# https://github.com/nlintz/TensorFlow-Tutorials/blob/master/05_convolutional_net.py

def init_weights(shape):
    return tf.Variable(tf.random_normal(shape, stddev=0.01))

w = init_weights([3, 3, 1, 32]) # filter 3*3*225, 32 outputs
w2 = init_weights([3, 3, 32, 64]) # filter 3*3*32, 64 outputs
w3 = init_weights([3, 3, 64, 128])
w4 = init_weights([128 * 4 * 4, 625])
w_o = init_weights([625, 3])

p_keep_conv = tf.placeholder("float")
p_keep_hidden = tf.placeholder("float")

l1a = tf.nn.relu(tf.nn.conv2d(x_train, w, strides = [1,1,1,1], padding = 'SAME'))
l1 = tf.nn.max_pool(l1a, ksize = [1, 2, 2, 1], strides = [1, 2, 2, 1], padding = 'SAME')
l1 = tf.nn.dropout(l1, p_keep_conv)

l2a = tf.nn.relu(tf.nn.conv2d(l1, w2, strides=[1, 1, 1, 1], padding='SAME'))
l2 = tf.nn.max_pool(l2a, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
l2 = tf.nn.dropout(l2, p_keep_conv)

l3a = tf.nn.relu(tf.nn.conv2d(l2, w3, strides=[1, 1, 1, 1], padding='SAME'))
l3 = tf.nn.max_pool(l3a, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
l3 = tf.reshape(l3, [-1, w4.get_shape().as_list()[0]])
l3 = tf.nn.dropout(l3, p_keep_conv)

l4 = tf.nn.relu(tf.matmul(l3, w4))
l4 = tf.nn.dropout(l4, p_keep_hidden)

py_x = tf.matmul(l4, w_o)

### ref.2

In [ ]:
# https://github.com/SphericalPotatoInVacuum/CNNDDDD/blob/master/cnn.py
def create_model():
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(filters=32, kernel_size=[5, 5], activation=tf.nn.relu,
                               input_shape=[64, 64, 225]),
        tf.keras.layers.MaxPool2D(pool_size=[2, 2], strides=2),
        tf.keras.layers.Conv2D(filters=64, kernel_size=[5, 5], activation=tf.nn.relu),
        tf.keras.layers.MaxPool2D(pool_size=[2, 2], strides=2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(1024, activation=tf.nn.relu),
        tf.keras.layers.Dropout(0.4),
        tf.keras.layers.Dense(10, activation=tf.nn.softmax)
    ])

    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    return model

### ref.3

In [ ]:
## 실험중-------------------------------------------------
# Convolutional 64 64 225 * 필터 3 3 225 16개 stride 1, padding 1 -> 64
W1 = tf.Variable(tf.truncated_normal(shape = [3, 3, 225, 64], stddev = 5e-2))
b1 = tf.Variable(tf.constant(0.1, shape = [64]))
print(W1.shape)
print(b1.shape)
# Activation Function ReLU
L1 = tf.nn.relu(tf.nn.conv2d(X, W1, strides = [1, 1, 1, 1], padding = 'SAME')+b1)
# Pooling 
L1 = tf.nn.max_pool(L1, ksize = [1, 2, 2, 1], strides = [1, 2, 2, 1], padding = 'SAME')
# Dropout
L1 = tf.nn.dropout(L1, keep_prob = keep_prob)

# FC

# CACAPCACAPCACAPF